# Draw dashboard to analyze dogami


## import lib

In [1]:

import pandas as pd
from pymongo import MongoClient
import plotly

## define functions

In [2]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [3]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df


## testing first


In [7]:
dog_df = read_mongo(db='dogami-database', collection='dog')

dog_df.head()

,token_id,generation,strength,intelligence,breeding_count,gender,size,status,friendliness,breed,group,obedience,rarity_tier,eyes_color,bonding_level,primary_personality,vitality,rarity_score,secondary_personality
0,122,Alpha,3.0,7.0,0.0,Female,Small female,Puppy,6.0,Labrador,Sporting,7,Silver,Brown #1,1.0,Dreamer,3.0,63.0,Foodie
1,56,Alpha,7.0,8.0,0.0,Female,Small female,Puppy,3.0,Toy Poodle,Non-Sporting,4,Diamond,Black #2,1.0,Talkative,8.0,91.0,Needy
2,32,Alpha,7.0,5.0,0.0,Female,Small female,Puppy,4.0,Toy Poodle,Non-Sporting,4,Silver,Brown #1,1.0,Independent,4.0,64.0,Emotional
3,126,Alpha,9.0,6.0,0.0,Male,Large male,Puppy,5.0,Husky,Working,3,Gold,Brown #1,1.0,Gentle,6.0,79.0,Funny
4,132,Alpha,3.0,3.0,0.0,Female,Large female,Puppy,8.0,French Bulldog,Non-Sporting,4,Bronze,Black #1,1.0,Lazy,5.0,34.0,Devoted


In [8]:
trans_df = read_mongo(db='dogami-database', collection='transaction')
trans_df = trans_df.to_numeric()
trans_df.head()

,token_id,operation_hash,price,time_stamp,order_type,trans_id,seller,buyer
0,7078,oo75o7qf3Lnw8aieRuxxM6YA4YU5YCwwBgYSY3UKCAgWid...,400,2022-03-16T21:14:14Z,sales,187856469,NaN,NaN
1,3974,oo3rxMSREHkGpmYcm6FKnjUbxoBYJZuLRjKNDxKTx3UunN...,277,2022-02-23T12:15:10Z,sales,177687229,NaN,NaN
2,3809,onwfL6mB9xgsnk15mKcpE7kAswRoUiHBDGejNTtNzfskf1...,233,2022-02-23T05:10:00Z,sales,177546335,NaN,NaN
3,2627,opLyxz8tY3s6cK43f1JB1sLqQstyTrdnL7ryMibfCLwBwp...,399,2022-03-07T08:14:24Z,sales,183337901,NaN,NaN
4,6499,opAJUzjQfCAJwnPf3Nq4tkJasomVYA88tqbAJqULJTN7Ty...,1999,2022-03-12T13:10:54Z,sales,185879660,NaN,NaN


In [10]:
trans_df.count()

token_id          2529
operation_hash    2529
price             2529
time_stamp        2529
order_type        2529
trans_id          2529
seller            1203
buyer             1203
dtype: int64

In [11]:
join_df = trans_df.join(other=dog_df, on='token_id', how='left')

join_df

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat